# Part 4: Distributed Data Parallel

So far, we have used only a single GPU for training. But you may have multiple GPUs available. How do you use them? That's what we do here.

Let's take a moment to think about what's going to happen in a multi-GPU setting. In a real world setting, you may have multiple boxes (nodes) and each of it may have multiple GPUs. You have your script for training the model. It's good to imagine it this way: several Python interpreters (depending on the total number of GPUs) are going to run the exact same script in parallel, without being aware of what the others are doing. So if you have a print statement in your script, all your GPUs are going to print it. Each GPU is going to run a *process* in parallel.

You generally have a way of identifying each GPU by a unique global ID (rank). You also get the ID of the GPU within that node. PyTorch gives this by setting some environment variables such as RANK and LOCAL_RANK. All GPUs run the exact same script, they only differ by these environment variables.

Now, your task as a programmer is to ensure that you setup your script in such a way that you can run the exact same script on multiple GPUs at the same time without running into issues. You need to compute loss properly. You also need to make sure that you are passing different data to each GPU. Also, you have no control over which of the GPUs will start / finish before which other GPU. You have to assume that you cannot predict this.

You typically assign the GPU with rank = 0 as the master process. For one time tasks, such as printing the loss, you generally print only if the GPU is the master process.

Lastly, you need a way to *destroy* these processes as well for cleaning up the mess you left.

In PyTorch, you can do this using distributed data parallel.

In [ ]:
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import os
import torch
import tiktoken
import math
import time

ddp = int(os.environ.get('RANK', -1)) != -1  # if you have multiple GPUs then, this condition will be true & you want to execute script in parallel

if ddp:
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE']) # the total number of GPUs across all nodes

    device = f'cuda:{ddp_local_rank}' # Device name within that node. All nodes index GPUs as cuda:0, cuda:1, etc. Thus, we use local rank
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # if the zeroth GPU, then this will be true. For checkpointint, logging, etc.

else:
    # non ddp run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'

In [ ]:
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

total_batch_size = 524288
B = 16
T = 1024

assert total_batch_size / (B * T * ddp_world_size) == 0, "make sure that total batch size is nicely divisible by the batches running"
grad_accum_steps = (total_batch_size) // (B * T * ddp_world_size) # Since there are multiple GPUs, each will have B * T micro batch running


if master_process:
    print(f"Total Desired Batch Size: {total_batch_size}")
    print(f"Number of Gradient Accumulation Steps: {grad_accum_steps}")

To execute the script on multiple GPUs in parallel, we cannot just do: `python script.py`. We have to do something like this:
```bash
torchrun --standalone --nproc_per_node=8 script.py
```

`nproc_per_node` depends on the number of GPUs you have in each of the box.

### DataLoader for Parallel Processing

We want each GPU to be processing different data and not the same batch. What we want is the batch sampling to be dependent on which GPU we are on- and to make it more generic, we want to make it dependent on the rank of the GPU and the total number of GPUs.

Basically, what we want to do is iterate over the dataset in blocks as follows:

Batch 1 For GPU1, Batch 1 for GPU2, Batch 1 for GPU3, Batch 2 for GPU1, Batch 2 for GPU2, Batch 2 for GPU3...

You can work out the exact indexing for this. It's pretty simple.

In [ ]:
class DataLoaderLite:
    def __init__(self, B, T, process_rank, num_processes):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes

        with open('input.txt', 'r') as f:
            text = f.read()

        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"Loaded {len(self.tokens)} tokens")

        # The starting position now depends on the GPU number
        self.current_position = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position*B*T*1]
        x = (buf[:-1]).view(B, T)
        y = (buf[1:]).view(B, T)

        # Advancing by keeping in mind the number of GPUs running
        self.current_position += B*T*self.num_processes

        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_position = self.B * self.T * self.process_rank

        return x, y
    
# if num_processes = 1 and process_rank = 0, then it's identical to what we had before.

In [ ]:
train_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size)

Next, we need to initialize the GPT model class. Each of the GPU will initialize the model now. We want all of their weights to be initialized to the same values. So we absolutely need to set the seed, which we have done above. Because of the seed, all the GPUs will initialize the same model, move it to the device, and compile it.

```python
model = GPT(GPTConfig(vocab_size=50304))
model.to(device)
model = torch.compile(model)

if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

raw_model = model.module if ddp else model
```

So in effect, if there are eight GPUs, you have eight identical models, and their compilations on each of the GPUs. 

NOTE: If we want to parallely process each models, we need to wrap the model in the DDP container class from PyTorch. In forward pass, nothing changes. But in the backward pass, each GPU has the gradients for the parameters. What DDP does for you is that when the backward pass is done, it will do a `AllReduce`, and average gradients across all the GPUs running, and then deposit this average at every GPU. ( For further optimization, DDP will dispatch AllReduce synchronization as the backward pass is happening ).

In [ ]:
model = None # assume the model has been initialized as in above cell

# LR scheduler stays as it is
max_steps = 50 
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps # (it + 1) to ensure we don't start at zero i.e. when it=0
    if it > max_steps:
        return min_lr
    
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)


Optimizer also doesn't change. But the training loop needs to change.

Think about this:

We are running micro batches on each GPU and while running the micro batches, we are accumulating gradients at each GPU. After each micro batch, if we synchronize and average gradients across the GPUs, then it's being wasteful. We would want DDP to average the gradients only when we are at the last micro batch i.e. when we complete one total batch. 

In PyTorch, the way to do this is by using the `@no_sync()` context manager, which currently sets the flag for whether to synchronize or not. 

In the tutorial, Andrej uses a hack which avoids using the context manager and directly updates the flag. We'll use the same here. 

Also, note that the `accumulated_loss` is GPU specific. So when we are printing, we are printing the GPU specific loss for each GPU. What we would want is to average the loss across the GPUs and then print only once.

In [ ]:
import torch.distributed as dist

# Assume that optimizer is loaded as before- using configure_optimizer method
# raw_model is defined above
raw_model = None
optimizer = raw_model.configure_optimizer() # model is now the DDP model. So we need to say raw_model.config_optim

for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    accumulated_loss = 0.0

    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)

        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)

        loss = loss / grad_accum_steps
        accumulated_loss += loss.detach()

        # NOTE: Changes here
        # Disable synchronization till total batch is processed
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)

        # Synchronize only when the total batch is processed
        loss.backward()

    # Take average of loss across GPUs. It will deposit this average on all "ranks"
    if ddp:
        # After this call, all the ranks will have the same accumulated loss which is the average
        dist.all_reduce(accumulated_loss, op=dist.ReduceOp.AVG)

    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping

    optimizer.step()
    torch.cuda.synchronize()

    t1 = time.time() 
    dt = t1 - t0

    # tokens processed is also higher now
    tokens_processed = train_loader.B * train_loader.T * grad_accum_steps * ddp_world_size
    tokens_per_sec = tokens_processed / dt

    if master_process:
        print(f"Step {step:4d} | Loss: {loss.item():.6f} | norm: {norm:.4f} | dt: {dt*1000:.2f}ms | tok/sec: {tokens_per_sec} | lr: {lr:4e}")


# Destroy the process groups so it doesn't complain
if ddp:
    destroy_process_group()

The loss may not exactly match with the loss you got with single GPUs. It's mostly because the boundary conditions with the batches. Some tokens may not get added in the batch because of the way we're resetting it. But it's fine. 